In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S7'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 2912
Total Tokens Found = 35362
Total Tokens Output = 35295
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 2912
Total Tokens= 35295
Unique Tokens= 7254
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 2912
Sorted Token Size = 7254
Clean Token Size = 3422
*Stop Replacements here
References Processed =  2912
Total Replacement Pairs = 186
Tokens Read = 35295
Tokens Changed =  268
References Changed = 252


In [2]:
status="done"

In [3]:
print(status)

done


In [14]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [4]:
import numpy as np

Create a function of the parameters that returns the F score

In [5]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [6]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 3660
NewMatrix Iterateblocks Total Time = 2.5997740095481277


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2746
NewMatrix Iterateblocks Total Time = 2.1355690099298954


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1718
NewMatrix Iterateblocks Total Time = 1.8506264770403504


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 551
NewMatrix Iterateblocks Total Time = 0.9212935985997319

Precision = TP / (TP + FP) ......................... = 0.27917889
Recall = TP / (TP + FN) ....................

0.43384475

Create functions for variable transformation.

In [7]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [8]:
inv_softplus(1)

0.541324854612918

In [9]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [10]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [11]:
import os, sys

In [12]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [13]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [14]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [15]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-0.2661815826681464

In [48]:
optimizer=None

In [49]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [50]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
)

In [51]:
optimizer.probe(
    params={"beta_sp": inv_softplus(14), 
            "sigma_sp": inv_softplus(24),
            "mu_logit":logit(0.5),
            #'epsilonIterate_logit': logit(0),
            "muIncr_logit":logit(0.1),
            "epsilon_sp":inv_softplus(3.0)},
    lazy=True,
)

In [52]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.50|Mu+:0.10|Beta:14.00|Sigma:24.00|Epsilon:3.00|EpIter:0.00
|  1        |  2.229    |  14.0     |  2.949    | -2.197    |  0.0      |  24.0     |
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  2        |  1.885    |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  3        |  1.671    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  4        |  0.8445   |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  5        | -0.7744   |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|Ep

|  39       |  0.8427   |  270.0    |  571.3    | -2.48     |  3.696    |  110.4    |
Mu:0.70|Mu+:0.03|Beta:282.00|Sigma:108.00|Epsilon:478.41|EpIter:0.00
|  40       |  0.9707   |  282.8    |  478.4    | -3.563    |  0.8478   |  108.2    |
Mu:0.62|Mu+:0.08|Beta:284.00|Sigma:96.00|Epsilon:574.98|EpIter:0.00
|  41       | -4.916    |  284.8    |  575.0    | -2.415    |  0.4691   |  96.84    |
Mu:0.95|Mu+:0.01|Beta:282.00|Sigma:104.00|Epsilon:489.00|EpIter:0.00
|  42       |  1.23     |  282.8    |  489.0    | -4.458    |  3.018    |  104.4    |
Mu:0.89|Mu+:0.01|Beta:13.00|Sigma:26.00|Epsilon:14.44|EpIter:0.00
|  43       |  1.771    |  13.7     |  14.44    | -4.494    |  2.06     |  26.3     |
Mu:0.97|Mu+:0.02|Beta:299.00|Sigma:108.00|Epsilon:500.34|EpIter:0.00
|  44       |  1.042    |  299.9    |  500.3    | -3.816    |  3.353    |  108.5    |
Mu:0.73|Mu+:0.02|Beta:264.00|Sigma:102.00|Epsilon:579.35|EpIter:0.00
|  45       |  1.505    |  264.3    |  579.4    | -4.096    |  0.9724   | 

In [53]:
optimizer.max

{'target': 2.2287101631971957,
 'params': {'beta_sp': 13.999999168470936,
  'epsilon_sp': 2.9489308190572983,
  'muIncr_logit': -2.197224577336219,
  'mu_logit': 0.0,
  'sigma_sp': 23.99999999996225}}

In [54]:
inv_logit(optimizer.max["target"])

0.90279823

In [55]:
params=optimizer.max["params"]

In [56]:
softplus(params["beta_sp"])

14.0

In [57]:
softplus(params["sigma_sp"])

24.0

In [58]:
inv_logit(params["mu_logit"])

0.5

In [59]:
inv_logit(params["muIncr_logit"])

0.10000000000000003

In [60]:
inv_logit(params["epsilonIterate_logit"])

KeyError: 'epsilonIterate_logit'

In [61]:
softplus(params["epsilon_sp"])

3.0

In [62]:
entropy_F1(mu=0.5,
           muIncr=0.1,
           beta=14,
           sigma=24,
           epsilon=3.0,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 3482
NewMatrix Iterateblocks Total Time = 1.592468279413879


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2843
NewMatrix Iterateblocks Total Time = 1.4765032278373837


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1930
NewMatrix Iterateblocks Total Time = 1.2834587404504418


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 870
NewMatrix Iterateblocks Total Time = 0.7057199785485864


>>>>>>>>>>>>
Starting Iteration mu= 0.8999999999999999
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Bl

0.9056188